In [1]:

import os
import pandas as pd

# !git clone https://github.com/ultralytics/yolov5  # clone
%cd /home/san/code/optimaize/yolov5/
# %pip install -qr requirements.txt  # install

import torch
import utils
# display = utils.notebook_init()  # checks

/home/san/code/optimaize/yolov5


/home/san/.pyenv/versions/3.9.7/envs/yolov5/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Video procesing
%cd /home/san/code/optimaize/yolov5/data/videos/
# reduce fisheye effect of gopro
!ffmpeg -i setup2.mp4 -ss 00:00:00.00 -t 00:00:20  -vf "rotate=+5 *PI/180, crop=in_w*4/5:in_h*2/3:in_w/5:in_h/3"  -y setup2_short.mp4



/home/san/code/optimaize/yolov5/data/videos
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolam

In [3]:
# !ffplay setup2_short.mp4

In [4]:
%cd /home/san/code/optimaize/yolov5/
!python detect.py --source ./data/videos/setup2_short.mp4 --save-txt --save-conf --classes 0 --name inovelec;

/home/san/code/optimaize/yolov5
detect: weights=yolov5s.pt, source=./data/videos/setup2_short.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=[0], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=inovelec, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.9.7 torch-1.12.0+cu102 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
video 1/1 (1/999) /home/san/code/optimaize/yolov5/data/videos/setup2_short.mp4: 320x640 5 persons, Done. (0.074s)
video 1/1 (2/999) /home/san/code/optimaize/yolov5/data/videos/setup2_short.mp4: 320x640 5 persons, Done. (0.070s)
video 1/1 (3/999) /home/san/code/optimaize/yolov5/data/videos/setup2_short.mp4: 320x640 5 persons, Done. (0.068s)
video 1/1 (4/999) /home/san/code/

/home/san/code/optimaize/yolov5


In [6]:
# Create dataframe from txt files
%cd /home/san/code/optimaize/yolov5/
path = os.path.join('.', 'runs', 'detect', 'inovelec', 'labels') #runs/detect/inovelec
filenames = next(os.walk(path), (None, None, []))[2]

stack = []
for file in filenames:
    # print(file[13:-4], end=' ')
    # Read txt file
    txt_file = open(os.path.join(path,file), "r")
    file_content = txt_file.read()
    txt_file.close()

    content_list = file_content.split("\n")
    first_split  = file_content.split("\n")
    second_split = [ ([file[13:-4]] + x.split(' ')) for x in first_split][:-1]    
    stack = stack + second_split
    
ino = pd.DataFrame(stack, columns = ['frame', 'class', 'xini', 'yini', 'xfin', 'yfin', 'score'])
ino = ino.astype({'frame':'int','class':'int', 'xini':'float', 'yini':'float', 'xfin':'float', 'yfin':'float', 'score':'float'})


In [7]:
ino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3527 entries, 0 to 3526
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   frame   3527 non-null   int64  
 1   class   3527 non-null   int64  
 2   xini    3527 non-null   float64
 3   yini    3527 non-null   float64
 4   xfin    3527 non-null   float64
 5   yfin    3527 non-null   float64
 6   score   3527 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 193.0 KB


In [8]:
ino

,frame,class,xini,yini,xfin,yfin,score
0,69,0,0.878125,0.488194,0.042188,0.087500,0.698782
1,69,0,0.731641,0.536111,0.041406,0.119444,0.706629
2,69,0,0.401563,0.556250,0.042188,0.143056,0.734309
3,69,0,0.354688,0.556944,0.042188,0.127778,0.783327
4,69,0,0.536719,0.858333,0.118750,0.275000,0.870552
...,...,...,...,...,...,...,...
3522,623,0,0.875391,0.484722,0.039844,0.091667,0.473757
3523,623,0,0.733203,0.536111,0.044531,0.122222,0.645748
3524,623,0,0.403516,0.552083,0.044531,0.143056,0.702861
3525,623,0,0.358984,0.559722,0.047656,0.127778,0.769290
